In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/COMMUNICATION SKILLS-1.pdf")
data = loader.load()

In [3]:
len(data) 
data[0]

Document(metadata={'source': 'data/COMMUNICATION SKILLS-1.pdf', 'page': 0}, page_content='GOKARAJU\x00RANGARAJUInstitute\x00of\x00Engineering\x00and\x00Technology\n(Autonomous)Dr. T. Jagannadha Swamy\nDean Career Guidance Compiled by \nDr. K.V.S. Raju\nDean T rainingCOMMUNICATION \nSKILLS\nSKILL\nSERIES')

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

texts = text_splitter.split_documents(data)

In [5]:
len(texts)


18

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

C:\Users\DELL 5300 A&I\AppData\Local\Temp\ipykernel_10220\3059872029.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


In [10]:
from langchain_pinecone import PineconeVectorStore as vectordb

# from dotenv import load_dotenv
# load_dotenv()
index_name = "new-index"
PINECONE_API_KEY = "af51e754-6b68-41c0-a97a-f2a96bc5617b"
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

vector_store = vectordb.from_documents(texts , embed_model , index_name = index_name)

In [11]:
from langchain.retrievers import SVMRetriever

text = [doc.page_content for doc in texts]
svm_retriever = SVMRetriever.from_texts(text, embed_model )



In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import HuggingFaceHub
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = str(os.getenv("hf_token"))


prompt_str = """  You are an AI language model assistent . your job is to generates five different versions of the user question 
    to retrieve relevant documents from vector databases . By generating different perspectives of user question , your goal is to 
    overcome some of the limitations of distance-based Similarity-search .
    provide these alternative questions seperated by new lines
  Question: {query}
"""
#prompt
prompt = ChatPromptTemplate.from_template(prompt_str)

#llm
llm = HuggingFaceHub(repo_id="meta-llama/Llama-3.2-1B")

#generate the list of queries
generate_query = prompt | llm | StrOutputParser() | (lambda x:x.split("/n"))
res=generate_query.invoke({'query':'how to communicate effectively ?'})
len(res)

1

In [24]:

#retrieve the list of queries from retriver

from langchain.load import dumps, loads

def get_unique_docs(documents: list[list]) -> list:
    """
    Get unique documents from a list of lists of documents.

    Args:
        documents (list[list]): A list of lists containing documents.

    Returns:
        list: A list of unique documents.
    """
    # Flatten the list of lists into a single list of documents
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    
    # Remove duplicates by converting to a set and back to a list
    unique_docs = list(set(flattened_docs))
    
    # Load the unique documents back to their original format
    return [loads(doc) for doc in unique_docs]

retrival_chain = generate_query | svm_retriever.map() | get_unique_docs

In [26]:
res = retrival_chain.invoke({'query': 'what is the basic principle of comunication?'})
len(res)

4

In [27]:
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
#final prompt
template = """answer the question according to the provided context:
{context}

Question: {query}
"""
prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = ({'context': retrival_chain,'query': itemgetter('query')} | prompt | llm | StrOutputParser())

In [34]:
res=final_rag_chain.invoke({'query':" what are the common mistakes poeple do in communication ?", 'context': retrival_chain})
res

"Human: answer the question according to the provided context:\n[Document(metadata={}, page_content='GOKARAJU\\x00RANGARAJU\\nInstitute\\x00of\\x00Engineering\\x00and\\x00Technology\\n(Autonomous)'), Document(metadata={}, page_content='10\\nAbility to pay attention to and effectiv ely interpret what \\nother people are saying. For effectiv e listening, the \\nfollowing important components are required.\\n\\uf0a4 Accent\\n\\uf0a4 Speech\\n\\uf0a4 Sentence pattern\\n\\uf0a4 GestureSkills R equired for Effectiv e Listening:'), Document(metadata={}, page_content='attention to the messages people are sending with \\ntheir facial expressions and movements.'), Document(metadata={}, page_content='GOKARAJU\\x00RANGARAJUInstitute\\x00of\\x00Engineering\\x00and\\x00Technology\\n(Autonomous)Dr. T. Jagannadha Swamy\\nDean Career Guidance Compiled by \\nDr. K.V.S. Raju\\nDean T rainingCOMMUNICATION \\nSKILLS\\nSKILL\\nSERIES')]\n\nQuestion:  what are the common mistakes poeple do in communication ?

In [ ]:
from fastapi import FastAPI
from langserve import add_routes
import uvicorn
import os

app = FastAPI(
    title="Simple rag application",
    version="1.0.0",
    description="this is a simple rag application for communicating with clients"
)
add_routes(app, final_rag_chain, path="/chain")

if __name__ == "__main__":
    import asyncio

    import nest_asyncio

    nest_asyncio.apply()

    uvicorn.run(app, host="127.0.0.1", port=3000)